In [1]:
import pandas as pd
import numpy as np

#treino = pd.read_csv('merged_treinofinal.csv')
treino = pd.read_csv('TH.csv')

In [2]:
treino['Data'] = pd.to_datetime(treino['Data']).dt.date

In [3]:
import datetime
#Horário de verão ou inverno
treino['Horario'] = 1

treino.loc[treino['Data'] < datetime.date(2023,3,26), 'Horario'] = 0
treino.loc[treino['Data'] < datetime.date(2022,10,30), 'Horario'] = 1
treino.loc[treino['Data'] < datetime.date(2022,3,27), 'Horario'] = 0
treino.loc[treino['Data'] < datetime.date(2021,10,31), 'Horario'] = 1
treino.loc[treino['Data'] < datetime.date(2021,3,28), 'Horario'] = 0
treino.loc[treino['Data'] < datetime.date(2020,10,25), 'Horario'] = 1
treino.loc[treino['Data'] < datetime.date(2020,3,29), 'Horario'] = 0
treino.loc[treino['Data'] < datetime.date(2019,10,27), 'Horario'] = 1
treino.loc[treino['Data'] < datetime.date(2019,3,31), 'Horario'] = 0
treino.loc[treino['Data'] < datetime.date(2018,10,28), 'Horario'] = 1

treino['hora_noite'] = pd.to_datetime(treino['hora_noite'])
treino.loc[treino['Horario'] == 1, 'hora_noite'] = treino.loc[treino['Horario'] == 1, 'hora_noite'] + pd.Timedelta(hours=1)
treino['hora_noite'] = treino['hora_noite'].dt.time

treino['Horário'] = pd.to_datetime(treino['Horário']).dt.time
treino['Noite'] = treino['Horário'] > treino['hora_noite']

In [4]:
treino['Noite'].value_counts()

False    923
True     301
Name: Noite, dtype: int64

In [5]:
treino['dif'] = treino['goals_home'] - treino['goals_away']
treino['xg_dif'] = treino['xG_home'] - treino['xG_away']

treino.loc[treino['dif']>0, 'result'] = 'H'
treino.loc[treino['dif']==0, 'result'] = 'D'
treino.loc[treino['dif']<0, 'result'] = 'A'

In [6]:
#Define o favorito
treino.loc[treino['B365H'] < 2.26, 'fav'] = 'H'
treino.loc[treino['B365H'] < 1.40, 'fav'] = 'HH'
treino.loc[treino['B365A'] < 2.26, 'fav'] = 'A'
treino.loc[treino['B365A'] < 1.40, 'fav'] = 'AA'
treino.loc[treino['fav'].isna(), 'fav'] = 'D'

In [ ]:
import matplotlib.pyplot as plt
order = ['H', 'D', 'A']
category_order = ['HH', 'H', 'D', 'A', 'AA']

# Convert 'x' column to categorical data type with the specified order
treino['fav'] = pd.Categorical(treino['fav'], categories=category_order, ordered=True)
treino['result'] = pd.Categorical(treino['result'], categories=order, ordered=True)

# Calculate the percentage of values in column 'y' for each unique value in column 'x'
result = treino.groupby('fav')['result'].value_counts(normalize=True) * 100

# Criar o gráfico de barras
result.unstack().plot(kind='bar',stacked=True)

# Configurar os rótulos dos eixos
plt.xlabel('Favorito')
plt.ylabel('Percentagem')

# Exibir o gráfico
plt.show()

In [ ]:
import seaborn as sns
#Fazer reverse ao cmap

order = ['H', 'D', 'A']

sns.heatmap(treino[['fav','B365H','B365D','B365A']].groupby('fav').mean().rename({'B365H': 'H','B365D': 'D', 'B365A': 'A'}, axis=1) - result.unstack()[order].reindex(['HH', 'H', 'D', 'A', 'AA']))

In [ ]:
treino[['Horário','result']].groupby('Horário').count()

In [ ]:
import requests

datas = treino['Data'].unique()

for data in datas:
    url = f'https://api.sunrise-sunset.org/json?lat=41.4500&lng=-8.3000&date={data}'
    response = requests.get(url)
    resposta = response.json()
    treino.loc[treino['Data'] == data, 'hora_noite'] = resposta['results']['astronomical_twilight_end']
    print(data)


In [ ]:
import requests

datas = treino['Data'].unique()

for data in datas:
    url = f'https://api.open-meteo.com/v1/forecast?latitude=41.45&longitude=8.3&daily=temperature_2m_max'
    response = requests.get(url)
    resposta = response.json()
    print(resposta)
    #treino.loc[treino['Data'] == data, 'temperatura'] = resposta['temperature_2m']
    #print(data)


    

In [ ]:
import matplotlib.pyplot as plt

# Calcular a contagem de ocorrências de cada valor por árbitro
# Contar o número de jogos arbitrados por árbitro
counts_arbitro = treino['Árbitro'].value_counts()

# Filtrar os árbitros com menos de 10 jogos arbitrados
arbitros_filtrados = counts_arbitro[counts_arbitro >= 10].index

# Filtrar o DataFrame original com base nos árbitros filtrados
df_filtrado = treino[treino['Árbitro'].isin(arbitros_filtrados)]

# Calcular a contagem de ocorrências de cada valor por árbitro no DataFrame filtrado
counts = df_filtrado.groupby('Árbitro')['result'].value_counts(normalize=True).mul(100).rename('Percentage').reset_index()

counts = counts[counts['result'] == 'H']

# Criar o gráfico de barras
plt.figure(figsize=(10, 6))
sns.barplot(x='Árbitro', y='Percentage', hue='result', data=counts)
plt.title('Percentagem de vitórias da equipa da casa por árbitro\n(Apenas considerados árbitros com mais de 10 jogos arbitrados)')
plt.xlabel('Árbitro')
plt.ylabel('Percentagem de ocorrências')
plt.xticks(rotation=90)
plt.legend(title='Resultado')

plt.show()

In [ ]:
# Testar se há evidência estatística de que a presença do árbitro João Bento influencia a favor da equipa da casa

bento = treino[treino['Árbitro'] == 'João Bento']
nao_bento = treino[treino['Árbitro'] != 'João Bento']

# -> H0: média da dif de golos com joão bento - média da dif de golos sem joão bento = 0 (A equipa da casa não tem vantagem)
# -> H1: média da dif de golos com joão bento - média da dif de golos sem joão bento > 0 (A equipa da casa tem vantagem)

test_statistic, p_value = ttest_ind(bento['dif'], nao_bento['dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: 1.93
# p-value: 0.0269

# Como p-value < 0.05, rejeitamos H0, o que nos diz que a equipa da casa é beneficiada com a presença do árbitro João Bento

# Nota: Este árbitro foi despromovido na época 2020/21

In [ ]:
# Testar se há evidência estatística de que a presença do árbitro António Nobre influencia a favor da equipa da casa

bento = treino[treino['Árbitro'] == 'António Nobre']
nao_bento = treino[treino['Árbitro'] != 'António Nobre']

# -> H0: média da dif de golos com joão bento - média da dif de golos sem joão bento = 0 (A equipa da casa não tem vantagem)
# -> H1: média da dif de golos com joão bento - média da dif de golos sem joão bento > 0 (A equipa da casa tem vantagem)

test_statistic, p_value = ttest_ind(bento['dif'], nao_bento['dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: 1.93
# p-value: 0.0269

# Como p-value < 0.05, rejeitamos H0, o que nos diz que a equipa da casa é beneficiada com a presença do árbitro João Bento

# Nota: Este árbitro foi despromovido na época 2020/21

In [ ]:
treino[treino['Árbitro'] == 'António Nobre']['dif'].mean()

In [ ]:
treino[treino['Árbitro'] != 'António Nobre']['dif'].mean()

In [ ]:
import seaborn as sns

sns.boxplot(x=treino['Assistance'])

In [ ]:
treino['Assistance'].describe()

In [ ]:
treino['dif'] = treino['goals_home'] - treino['goals_away']
treino['xg_dif'] = treino['xG_home'] - treino['xG_away']

treino_t1 = treino[['Data','Assistance','dif','xg_dif']].copy()

covid = treino_t1[treino_t1['Assistance'] == 0]
nao_covid = treino_t1[treino_t1['Assistance'] != 0]

In [ ]:
# -> H0: média da dif de golos com espectadores - média da dif de golos sem espectadores = 0
# -> H1: média da dif de golos com espectadores - média da dif de golos sem espectadores > 0
from scipy.stats import ttest_ind

test_statistic, p_value = ttest_ind(covid['dif'], nao_covid['dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: -0.72
# p-value: 0.7645

# Não rejeitamos a hipotese nula, a presença de espectadores não afeta a diferença de golos

In [ ]:
# -> H0: média da dif de golos com espectadores - média da dif de golos sem espectadores = 0
# -> H1: média da dif de golos com espectadores - média da dif de golos sem espectadores > 0
from scipy.stats import ttest_ind


test_statistic, p_value = ttest_ind(covid['xg_dif'], nao_covid['xg_dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: -0.67
# p-value: 0.7472

# Não rejeitamos a hipotese nula

In [ ]:
treino.loc[treino['Dia'] == 'sáb', 'FDSem'] = 1
treino.loc[treino['Dia'] == 'dom', 'FDSem'] = 1
treino.loc[treino['FDSem'].isna(), 'FDSem'] = 0

In [ ]:
treino['Noite'].value_counts()

In [ ]:
fim = treino[treino['FDSem'] == 1]
nao_fim = treino[treino['FDSem'] != 1]

noite = treino[treino['Noite'] == True]
nao_noite = treino[treino['Noite'] == False]

In [ ]:
# -> H0: média da dif de golos com jogo ao fim de semana
# -> H1: média da dif de golos com jogo durante a semana
from scipy.stats import ttest_ind


test_statistic, p_value = ttest_ind(fim['xg_dif'], nao_fim['xg_dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: 1.46
# p-value: 0.0722

In [ ]:
# -> H0: média da dif de golos com jogo à noite
# -> H1: média da dif de golos com jogo durante o dia
from scipy.stats import ttest_ind


test_statistic, p_value = ttest_ind(noite['xg_dif'], nao_noite['xg_dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: -0.03
# p-value: 0.5101

In [ ]:
treino['formacao_home'].value_counts()

In [14]:
treino[treino['formacao_home'].isna()]

,Jornada,Dia,Data,Horário,home,xG_home,xG_away,away,Assistance,Local,...,jogador19_home,jogador20_away,jogador20_home,fav,dif,xg_dif,result,hora_noite,Horario,Noite
1020,12.0,sáb,2021-11-27,20:30:00,Belenenses,0.0,3.6,Benfica,6096,Estádio Nacional,...,NaN,NaN,NaN,AA,-7,-3.6,A,18:43:42,0,True
1021,12.0,dom,2021-11-28,15:30:00,Maritimo,0.3,1.3,Pacos Ferreira,2017,Estádio dos Barreiros,...,NaN,NaN,NaN,D,2,-1.0,H,18:43:27,0,False
1022,12.0,dom,2021-11-28,18:00:00,Sp Lisbon,2.1,0.3,Tondela,34058,Estádio José Alvalade,...,NaN,NaN,NaN,HH,2,1.8,H,18:43:27,0,False
1023,12.0,dom,2021-11-28,20:30:00,Porto,3.9,1.5,Guimaraes,26407,Estádio Do Dragão,...,NaN,NaN,NaN,HH,1,2.4,H,18:43:27,0,True
1024,12.0,seg,2021-11-29,20:15:00,Estoril,1.7,1.6,Santa Clara,658,Estádio António Coimbra da Mota,...,NaN,NaN,NaN,H,0,0.1,D,18:43:13,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,34.0,sáb,2022-05-14,18:00:00,Porto,1.6,0.7,Estoril,47512,Estádio Do Dragão,...,NaN,NaN,NaN,HH,2,0.9,H,22:38:29,1,False
1220,34.0,sáb,2022-05-14,18:00:00,Maritimo,1.5,1.3,Portimonense,4124,Estádio dos Barreiros,...,NaN,NaN,NaN,H,-1,0.2,A,22:38:29,1,False
1221,34.0,sáb,2022-05-14,20:30:00,Sp Lisbon,1.5,0.9,Santa Clara,28942,Estádio José Alvalade,...,NaN,NaN,NaN,HH,4,0.6,H,22:38:29,1,False
1222,34.0,dom,2022-05-15,18:00:00,Famalicao,2.7,0.8,Sp Braga,4739,Estádio Municipal 22 de Junho,...,NaN,NaN,NaN,A,1,1.9,H,22:40:05,1,False


In [105]:
seis = treino[treino['AST'] >= 6]
nao_seis = treino[treino['AST'] < 6]



In [106]:
# -> H0: média da dif de golos com jogo à noite
# -> H1: média da dif de golos com jogo durante o dia
from scipy.stats import ttest_ind


test_statistic, p_value = ttest_ind(seis['xg_dif'], nao_seis['xg_dif'], alternative='greater')

# Print the test statistic and p-value
print("t-test statistic: {:.2f}".format(test_statistic))
print("p-value: {:.4f}".format(p_value))

# t-test statistic: -0.03
# p-value: 0.5101

# tenho de corrigir

t-test statistic: 18.76
p-value: 0.0000
